<a href="https://colab.research.google.com/github/NadezhdaMalysheva/projects/blob/main/Experiments_with_hyper%20parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Подгружаем необходимые библиотеки 

In [1]:
pip install pytorch-tabnet

In [2]:
import numpy as np
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls

%matplotlib inline

In [4]:
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import joblib
from tqdm.notebook import tqdm

from pytorch_tabnet.tab_model import TabNetClassifier

from hyperopt import hp
import numpy as np

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

import lightgbm as lgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

from time import time
import joblib

In [5]:
import torch

#Загружаем данные, параметры

In [6]:
target = 'class'

add_columns = ['subClass', 'objID', 'z', 'zErr', 'ra', 'dec']

photo_columns = ['psfMag_u',	'psfMag_g',	'psfMag_r',	'psfMag_i',	'psfMag_z',
                 'cModelMag_u',	'cModelMag_g',	'cModelMag_r',	'cModelMag_i',	'cModelMag_z']

feature_columns = (
    photo_columns + add_columns + [target])

In [7]:
#df = joblib.load('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/df_agg.pkl')#pd.read_csv('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/df_agg.csv')
#agr_feature = [x for x in df.columns if x not in feature_columns]

In [8]:
def data_preparation(X, y, c=10000, test_size = 0.8):

    X1_train, X1_test, y1_train, y1_test = train_test_split(X[y==1], y[y==1], test_size=test_size, random_state = 43)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X[y==2], y[y==2], test_size=test_size, random_state = 43)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X[y==3], y[y==3], test_size=test_size, random_state = 43)
    
    count = c
    count1 = c

    X_train, X_test = np.concatenate((X1_train[:count], X2_train[:count], X3_train[:count])), np.concatenate((X1_test[:count1], X2_test[:count1], X3_test[:count1]))
    y_train, y_test = np.concatenate((y1_train[:count], y2_train[:count], y3_train[:count])), np.concatenate((y1_test[:count1], y2_test[:count1], y3_test[:count1]))
    

    return [X_train, X_test, y_train, y_test, X1_train, X1_test, y1_train, y1_test, X2_train, X2_test, y2_train, y2_test, X3_train, X3_test, y3_train, y3_test]

In [9]:
def scor(y_test, y_pred):
  return accuracy_score(y_test, y_pred)

lgb_reg_params = {
    'learning_rate':    hp.uniform('learning_rate', 0.001, 0.1),
    'min_child_samples':hp.randint('min_child_samples', 50)+1,
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 0.9),
    'num_leaves' :      hp.randint('num_leaves', 100)+10,
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.99),
    'n_estimators':     hp.randint('n_estimators', 1500)+100
}
lgb_fit_params = {
    'early_stopping_rounds': 20,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['score'] = lambda y, pred: -accuracy_score(y, pred)


rf_reg_params = {
    'min_samples_leaf': hp.randint('min_samples_leaf', 20)+1,
    'min_samples_split':hp.uniform('min_samples_split', 0.001, 0.1),
    #'max_features':     hp.choice('max_features', ['auto', 'sqrt', 'log2', None]),
    #'learning_rate':    hp.uniform('learning_rate', 0.001, 0.1),
    'n_estimators':     hp.randint('n_estimators', 800)+100
}
rf_fit_params = {
}
rf_para = dict()
rf_para['reg_params'] = rf_reg_params
rf_para['fit_params'] = rf_fit_params
rf_para['score'] = lambda y, pred: -accuracy_score(y, pred)

tabnet_reg_params = {
    'n_d' :              64,
    'n_a' :              64,
    'n_steps' :          hp.randint('n_steps', 10-3)+3,
    'gamma' :            hp.uniform('gamma', 1.0, 3.0),
    'lambda_sparse' :    hp.uniform('lambda_sparse', 0.0, 0.01),
    'momentum' :         0.3, 
    'clip_value' :       2.,
    'optimizer_params' : dict(lr=2e-2),
    'scheduler_params' : {"step_size":50, "gamma":0.9},
    'scheduler_fn' :     torch.optim.lr_scheduler.StepLR,
    'mask_type' :       'entmax'
}

tabnet_fit_params = {
    'max_epochs' : 100, 
    'patience' : 15,
    'batch_size' : 512,
    'virtual_batch_size' : 128,
    'num_workers' : 0,
    'weights' : 1,
    'drop_last' : False,
    #'from_unsupervised' : unsupervised_model
}
tabnet_para = dict()
tabnet_para['reg_params'] = tabnet_reg_params
tabnet_para['fit_params'] = tabnet_fit_params
tabnet_para['score'] = lambda y, pred: -accuracy_score(y, pred)

In [10]:
target = 'class'

add_columns = ['subClass', 'objID', 'z', 'zErr', 'ra', 'dec']

photo_columns = ['psfMag_u',	'psfMag_g',	'psfMag_r',	'psfMag_i',	'psfMag_z',
                 'cModelMag_u',	'cModelMag_g',	'cModelMag_r',	'cModelMag_i',	'cModelMag_z']

feature_columns = (
    photo_columns + add_columns + [target])

In [11]:
X, y  = joblib.load('/content/drive/MyDrive/Научная работа/Спецсем/TabNetModel/X_y.pkl')

# Общая функция

In [ ]:
#dftn_time = pd.DataFrame(columns = ['Rows', 'train', 'test', 'Nd',	'Na',	'B',	'BV',	'mB',	'λsparse',	'Nsteps',	'γ',	'learning rate',	'decay rate',	'decay iterations', 'shared', 	'decision',	'mask_type',	'accuracy_tn', 'time_learn_tn',	'time_tn',	'accuracy_gb', 'time_learn_gb',	'time_gb'])

In [12]:
dftn_time = pd.read_csv('/content/drive/MyDrive/Научная работа/Data/hyper/time.csv')

In [13]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


##Старые

In [ ]:
with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'w') as f:
  pass

In [ ]:
def bootstrap_accuracy(model, X_test, y_test):
  from sklearn.utils import resample
  from matplotlib import pyplot
  values = np.concatenate((X_test, y_test.reshape((len(y_test), 1))), axis=1)
  n_iterations = 100
  n_size = int(len(y_test) * 0.50)
  stats = list()
  for i in range(n_iterations):
    test = resample(values, n_samples=n_size)
    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    stats.append(score)
  #pyplot.hist(stats, range=(0.872, 0.9))
  #pyplot.show()
  alpha = 0.97
  p = ((1.0-alpha)/2.0) * 100
  lower = max(0.0, np.percentile(stats, p))
  p = (alpha+((1.0-alpha)/2.0)) * 100
  upper = min(1.0, np.percentile(stats, p))
  main = np.mean(stats)
  #print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))
  return main, np.max([main-lower, upper-main])

In [ ]:
def bootstrap(X_test, y_test, X_valid, y_valid):
  from sklearn.utils import resample
  from matplotlib import pyplot
  values = np.concatenate((X_test, y_test.reshape((len(y_test), 1))), axis=1)
  n_iterations = 100
  n_size = int(len(y_test) * 0.50)
  stats = list()
  for i in range(n_iterations):
    train_ind = np.random.randint(0, len(values), n_size)
    test_ind = np.setdiff1d(range(len(values)), train_ind)
    train = values[train_ind]
    test = values[test_ind]
    # fit model
    model = lgb.LGBMClassifier()
    model.fit(train[:,:-1], train[:,-1], eval_set=[(train[:,:-1], train[:,-1]), (X_valid, y_valid)], **lgb_fit_params)
    # evaluate model
    predictions = model.predict(test[:,:-1])
    test = resample(values, n_samples=n_size)
    predictions = model.predict(test[:,:-1])
    score = accuracy_score(test[:,-1], predictions)
    stats.append(score)
  alpha = 0.97
  p = ((1.0-alpha)/2.0) * 100
  lower = max(0.0, np.percentile(stats, p))
  p = (alpha+((1.0-alpha)/2.0)) * 100
  upper = min(1.0, np.percentile(stats, p))
  main = np.mean(stats)
  #print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))
  return main, np.max([main-lower, upper-main])

In [ ]:
def feature_acc(model, str_m, Rows):
  print(str_m)

  feature_imp=pd.DataFrame((zip(model.feature_importances_, photo_columns+agr_feature)), columns=['Model','Feature'])
  t = 3
  feature = feature_imp.sort_values(by='Model', ascending=False).iloc[:t]['Feature'].values
  X = df[feature].values

  data_split = data_preparation(X, y, Rows, 0.8)
  count = Rows//3

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]
  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)

  X_train, y_train = train[:,:-1], train[:,-1].astype('int')
  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_valid   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  y_valid   = np.concatenate((y1_test[count : 2*count], y2_test[count : 2*count], y3_test[count : 2*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)

  acc, err = bootstrap(X_test_norm, y_test, X_valid_norm, y_valid)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc_feature '+str_m+': '+str(acc)+'+-'+str(err)+', ')

In [ ]:
def ones(number_exp, Rows, Nd,	Na,	B,	BV,	mB,	λsparse,	Nsteps,	γ, learning_rate,	decay_rate,	decay_iterations,	shared, decision, mask_type):
  
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('number: '+str(number_exp)+', ')
  
  #data
  data_split = data_preparation(X, y, c=Rows//3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000

  patience=50
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_test, y_test)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=patience,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time TN: '+str(t)+', ')

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 1000,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time GB: '+str(t)+', ')

  #Accuracy
  acc, err = bootstrap_accuracy(tn, X_test, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc TN: '+str(acc)+'+-'+str(err)+', ')
  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(tn, 'TN', 9000)
  feature_acc(gb, 'GB', 9000)

  #save model

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.txt')
  tn.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/tn'+str(number_exp))

In [ ]:
def memory(number_exp, Rows, Nd,	Na,	B,	BV,	mB,	λsparse,	Nsteps,	γ, learning_rate,	decay_rate,	decay_iterations,	shared, decision, mask_type):
  
  #data
  data_split = data_preparation(X, y, c=Rows//3, test_size=0.5)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  #X_valid      = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  #y_valid      = np.concatenate((y1_test[count : 2*count], y2_test[count : 2*count], y3_test[count : 2*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  #X_valid_norm = robust.transform(X_valid)


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000

  patience=50
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_test, y_test)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=patience,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time TN: '+str(t)+', ')

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 1000,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_test_norm, y_test)],  **lgb_fit_params)
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time GB: '+str(t)+', ')

  #save model

  joblib.dump(gb, '/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.pkl')
  joblib.dump(tn, '/content/drive/MyDrive/Научная работа/Data/hyper/tn'+str(number_exp)+'.pkl')


##Новая

In [14]:
X_main_test = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/X_main_test.pkl')
y_main_test = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/y_main_test.pkl')


In [15]:
def time_model(number_exp, Rows, Nd,	Na,	B,	BV,	mB,	λsparse,	Nsteps,	γ, learning_rate,	decay_rate,	decay_iterations,	shared, decision, mask_type):

  global dftn_time
  global X_main_test 
  global y_main_test

  data = {'Rows':Rows, 'Nd':Nd,	'Na':Na,	'B':B,	'BV':BV,	'mB':mB,	'λsparse':λsparse,	'Nsteps':Nsteps,	'γ':γ,	'learning rate':learning_rate,	'decay rate':decay_rate,	'decay iterations':decay_iterations,	'shared':shared, 	'decision':decision,	'mask_type':mask_type}
  'accuracy_tn', 'time_learn_tn',	'time_tn',	'accuracy_gb', 'time_learn_tn',	'time_gb'
  data_split = data_preparation(X, y, c=Rows//3, test_size=0.5)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  data['train'] = len(y_train)
  data['test'] = len(y_test)

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_valid = X_test
  y_valid = y_test

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_main_test_norm = robust.transform(X_main_test)
  X_valid_norm = X_test_norm


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000

  patience=50
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=patience,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  data['time_learn_tn'] = t

  t = time()
  pred = tn.predict(X_main_test)
  t = time() - t
  data['time_tn'] = t
  print('Confusion Matrix: \n', confusion_matrix(y_main_test, pred))
  acc = accuracy_score(y_test, tn.predict(X_test))
  data['accuracy_tn'] = acc
  print('Testing Score: ', acc)

  gb = lgb.LGBMClassifier( **{'n_estimators': 1000})

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  data['time_learn_gb'] = t

  t = time()
  pred = gb.predict(X_main_test_norm)
  t = time() - t
  data['time_gb'] = t
  print('Confusion Matrix: \n', confusion_matrix(y_main_test, pred))
  acc = accuracy_score(y_test, gb.predict(X_test_norm))
  data['accuracy_gb'] = acc
  print('Testing Score: ', acc)
  #save model
  print(data)
  dftn_time = dftn_time.append(data, ignore_index=True)
  dftn_time.to_csv('/content/drive/MyDrive/Научная работа/Data/hyper/time.csv', index=False)

  joblib.dump(gb, '/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.pkl')
  joblib.dump(tn, '/content/drive/MyDrive/Научная работа/Data/hyper/53/tn'+str(number_exp)+'.pkl')


#Запускаем тесты по времени (в данном эксперименте не так важна точность, она будет проверяться дальше)

##9000 rows

In [ ]:
time_model(number_exp=1.1, 
     Rows=9000, 
     Nd=8,	Na=8,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=1, decision=2, 
     mask_type='entmax')

Device used : cpu
epoch 0  | loss: 0.99334 | train_logloss: 9.61676 | train_accuracy: 0.248   | valid_logloss: 9.46599 | valid_accuracy: 0.24978 |  0:00:01s
epoch 1  | loss: 0.68227 | train_logloss: 8.78314 | train_accuracy: 0.48922 | valid_logloss: 8.69884 | valid_accuracy: 0.50467 |  0:00:03s
epoch 2  | loss: 0.56358 | train_logloss: 15.50828| train_accuracy: 0.26867 | valid_logloss: 15.43422| valid_accuracy: 0.272   |  0:00:04s
epoch 3  | loss: 0.49315 | train_logloss: 8.84363 | train_accuracy: 0.43444 | valid_logloss: 9.04509 | valid_accuracy: 0.42233 |  0:00:06s
epoch 4  | loss: 0.44271 | train_logloss: 8.63715 | train_accuracy: 0.36622 | valid_logloss: 8.71142 | valid_accuracy: 0.37589 |  0:00:07s
epoch 5  | loss: 0.42997 | train_logloss: 22.82182| train_accuracy: 0.09344 | valid_logloss: 22.8533 | valid_accuracy: 0.09333 |  0:00:09s
epoch 6  | loss: 0.406   | train_logloss: 8.37036 | train_accuracy: 0.17756 | valid_logloss: 8.27681 | valid_accuracy: 0.181   |  0:00:10s
epoch 7  

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


In [ ]:
time_model(number_exp=2, 
     Rows=9000, 
     Nd=16,	Na=16,
     B=512,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.82143 | train_logloss: 6.32878 | train_accuracy: 0.28522 | valid_logloss: 6.27567 | valid_accuracy: 0.28133 |  0:00:01s
epoch 1  | loss: 0.47692 | train_logloss: 3.73853 | train_accuracy: 0.46722 | valid_logloss: 3.60361 | valid_accuracy: 0.47044 |  0:00:02s
epoch 2  | loss: 0.43349 | train_logloss: 2.93796 | train_accuracy: 0.38289 | valid_logloss: 2.93365 | valid_accuracy: 0.38278 |  0:00:04s
epoch 3  | loss: 0.40709 | train_logloss: 8.79966 | train_accuracy: 0.33711 | valid_logloss: 8.77917 | valid_accuracy: 0.32978 |  0:00:05s
epoch 4  | loss: 0.3973  | train_logloss: 1.78235 | train_accuracy: 0.59833 | valid_logloss: 1.7947  | valid_accuracy: 0.60733 |  0:00:07s
epoch 5  | loss: 0.37775 | train_logloss: 1.26544 | train_accuracy: 0.40322 | valid_logloss: 1.28292 | valid_accuracy: 0.39467 |  0:00:08s
epoch 6  | loss: 0.37014 | train_logloss: 1.67077 | train_accuracy: 0.42756 | valid_logloss: 1.68211 | valid_accuracy: 0.42844 |  0:00:09s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063


In [ ]:
time_model(number_exp=3, 
          Rows=9000, 
          Nd=64,	Na=64,	
          B=512,	BV=128,	mB=0.7,	
          λsparse=0.001,	Nsteps=5,	γ=1.7, 
          learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
          shared=2, decision=2, 
          mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.08905 | train_logloss: 18.61257| train_accuracy: 0.369   | valid_logloss: 18.53504| valid_accuracy: 0.36978 |  0:00:02s
epoch 1  | loss: 0.71233 | train_logloss: 20.00418| train_accuracy: 0.29611 | valid_logloss: 20.22298| valid_accuracy: 0.289   |  0:00:04s
epoch 2  | loss: 0.52017 | train_logloss: 10.20336| train_accuracy: 0.40122 | valid_logloss: 10.19558| valid_accuracy: 0.39778 |  0:00:06s
epoch 3  | loss: 0.4706  | train_logloss: 10.50671| train_accuracy: 0.29522 | valid_logloss: 10.48509| valid_accuracy: 0.29267 |  0:00:08s
epoch 4  | loss: 0.42457 | train_logloss: 2.085   | train_accuracy: 0.38533 | valid_logloss: 2.11969 | valid_accuracy: 0.37678 |  0:00:10s
epoch 5  | loss: 0.39834 | train_logloss: 2.38837 | train_accuracy: 0.28467 | valid_logloss: 2.38355 | valid_accuracy: 0.28244 |  0:00:12s
epoch 6  | loss: 0.3888  | train_logloss: 4.24199 | train_accuracy: 0.29911 | valid_logloss: 4.28773 | valid_accuracy: 0.29133 |  0:00:14s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554


In [ ]:
time_model(number_exp=4, 
     Rows=9000, 
     Nd=32,	Na=32,
     B=512,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=3, decision=3, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.79962 | train_logloss: 10.63308| train_accuracy: 0.40778 | valid_logloss: 11.09464| valid_accuracy: 0.39244 |  0:00:01s
epoch 1  | loss: 0.47858 | train_logloss: 5.28805 | train_accuracy: 0.21689 | valid_logloss: 5.14806 | valid_accuracy: 0.22322 |  0:00:03s
epoch 2  | loss: 0.41724 | train_logloss: 2.92017 | train_accuracy: 0.32989 | valid_logloss: 2.87568 | valid_accuracy: 0.33378 |  0:00:05s
epoch 3  | loss: 0.39143 | train_logloss: 3.39911 | train_accuracy: 0.42067 | valid_logloss: 3.40527 | valid_accuracy: 0.41289 |  0:00:07s
epoch 4  | loss: 0.39031 | train_logloss: 3.75939 | train_accuracy: 0.36456 | valid_logloss: 3.75492 | valid_accuracy: 0.36511 |  0:00:08s
epoch 5  | loss: 0.36455 | train_logloss: 7.18896 | train_accuracy: 0.33311 | valid_logloss: 7.22629 | valid_accuracy: 0.33367 |  0:00:10s
epoch 6  | loss: 0.34964 | train_logloss: 4.84895 | train_accuracy: 0.34778 | valid_logloss: 4.85696 | valid_accuracy: 0.34989 |  0:00:12s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297


In [ ]:
time_model(number_exp=5, 
     Rows=9000, 
     Nd=128,	Na=128,	
     B=512,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.5721  | train_logloss: 15.15309| train_accuracy: 0.48111 | valid_logloss: 15.12557| valid_accuracy: 0.48756 |  0:00:02s
epoch 1  | loss: 0.74351 | train_logloss: 6.50309 | train_accuracy: 0.447   | valid_logloss: 6.47254 | valid_accuracy: 0.44122 |  0:00:04s
epoch 2  | loss: 0.67446 | train_logloss: 6.73639 | train_accuracy: 0.304   | valid_logloss: 6.9005  | valid_accuracy: 0.29444 |  0:00:06s
epoch 3  | loss: 0.50327 | train_logloss: 7.71363 | train_accuracy: 0.33056 | valid_logloss: 7.75076 | valid_accuracy: 0.33011 |  0:00:08s
epoch 4  | loss: 0.43796 | train_logloss: 3.02725 | train_accuracy: 0.29711 | valid_logloss: 3.03408 | valid_accuracy: 0.29356 |  0:00:10s
epoch 5  | loss: 0.42911 | train_logloss: 2.1664  | train_accuracy: 0.25989 | valid_logloss: 2.17608 | valid_accuracy: 0.25667 |  0:00:12s
epoch 6  | loss: 0.41588 | train_logloss: 3.1766  | train_accuracy: 0.35911 | valid_logloss: 3.1663  | valid_accuracy: 0.35589 |  0:00:14s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218


## 30000 rows

In [ ]:
time_model(number_exp=6, 
     Rows=30000, 
     Nd=8,	Na=8,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=1, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.70132 | train_logloss: 15.17457| train_accuracy: 0.35347 | valid_logloss: 15.3099 | valid_accuracy: 0.3517  |  0:00:01s
epoch 1  | loss: 0.42177 | train_logloss: 7.30538 | train_accuracy: 0.18657 | valid_logloss: 7.29039 | valid_accuracy: 0.19063 |  0:00:02s
epoch 2  | loss: 0.37971 | train_logloss: 4.83377 | train_accuracy: 0.3335  | valid_logloss: 4.8499  | valid_accuracy: 0.33347 |  0:00:04s
epoch 3  | loss: 0.35056 | train_logloss: 2.52646 | train_accuracy: 0.2967  | valid_logloss: 2.52153 | valid_accuracy: 0.30157 |  0:00:05s
epoch 4  | loss: 0.33972 | train_logloss: 1.27628 | train_accuracy: 0.45773 | valid_logloss: 1.27142 | valid_accuracy: 0.4559  |  0:00:06s
epoch 5  | loss: 0.33557 | train_logloss: 1.44308 | train_accuracy: 0.6143  | valid_logloss: 1.45259 | valid_accuracy: 0.60827 |  0:00:08s
epoch 6  | loss: 0.32793 | train_logloss: 1.46689 | train_accuracy: 0.5593  | valid_logloss: 1.4742  | valid_accuracy: 0.55867 |  0:00:09s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480


In [ ]:
time_model(number_exp=7, 
     Rows=30000, 
     Nd=16,	Na=16,
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.76376 | train_logloss: 8.69401 | train_accuracy: 0.34033 | valid_logloss: 8.49261 | valid_accuracy: 0.34137 |  0:00:01s
epoch 1  | loss: 0.4436  | train_logloss: 7.67326 | train_accuracy: 0.38297 | valid_logloss: 7.70181 | valid_accuracy: 0.38297 |  0:00:03s
epoch 2  | loss: 0.38025 | train_logloss: 5.99814 | train_accuracy: 0.37207 | valid_logloss: 5.98659 | valid_accuracy: 0.37387 |  0:00:04s
epoch 3  | loss: 0.36162 | train_logloss: 2.36577 | train_accuracy: 0.2892  | valid_logloss: 2.37139 | valid_accuracy: 0.29147 |  0:00:06s
epoch 4  | loss: 0.34708 | train_logloss: 1.52415 | train_accuracy: 0.3922  | valid_logloss: 1.53449 | valid_accuracy: 0.3895  |  0:00:07s
epoch 5  | loss: 0.33675 | train_logloss: 1.93036 | train_accuracy: 0.42287 | valid_logloss: 1.92694 | valid_accuracy: 0.4237  |  0:00:09s
epoch 6  | loss: 0.33209 | train_logloss: 1.3011  | train_accuracy: 0.43963 | valid_logloss: 1.31314 | valid_accuracy: 0.43527 |  0:00:10s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578


In [ ]:
time_model(number_exp=8, 
          Rows=30000, 
          Nd=64,	Na=64,	
          B=2048,	BV=512,	mB=0.7,	
          λsparse=0.001,	Nsteps=5,	γ=1.7, 
          learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
          shared=2, decision=2, 
          mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.9988  | train_logloss: 18.71415| train_accuracy: 0.4233  | valid_logloss: 18.66625| valid_accuracy: 0.42407 |  0:00:02s
epoch 1  | loss: 0.55887 | train_logloss: 21.24174| train_accuracy: 0.33337 | valid_logloss: 21.30052| valid_accuracy: 0.33323 |  0:00:04s
epoch 2  | loss: 0.46631 | train_logloss: 21.49563| train_accuracy: 0.14713 | valid_logloss: 21.30088| valid_accuracy: 0.15167 |  0:00:06s
epoch 3  | loss: 0.38887 | train_logloss: 18.99009| train_accuracy: 0.3336  | valid_logloss: 18.99406| valid_accuracy: 0.3338  |  0:00:08s
epoch 4  | loss: 0.38343 | train_logloss: 5.08895 | train_accuracy: 0.3337  | valid_logloss: 5.09685 | valid_accuracy: 0.33367 |  0:00:10s
epoch 5  | loss: 0.38242 | train_logloss: 2.6987  | train_accuracy: 0.32787 | valid_logloss: 2.72169 | valid_accuracy: 0.32823 |  0:00:12s
epoch 6  | loss: 0.37912 | train_logloss: 7.61661 | train_accuracy: 0.27283 | valid_logloss: 7.67888 | valid_accuracy: 0.2721  |  0:00:14s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216


In [ ]:
time_model(number_exp=9, 
     Rows=30000, 
     Nd=32,	Na=32,
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=3, decision=3, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.78322 | train_logloss: 17.05534| train_accuracy: 0.3178  | valid_logloss: 17.05851| valid_accuracy: 0.3176  |  0:00:01s
epoch 1  | loss: 0.44694 | train_logloss: 7.00825 | train_accuracy: 0.31507 | valid_logloss: 6.94097 | valid_accuracy: 0.31543 |  0:00:03s
epoch 2  | loss: 0.3768  | train_logloss: 9.24496 | train_accuracy: 0.37033 | valid_logloss: 9.28357 | valid_accuracy: 0.3719  |  0:00:05s
epoch 3  | loss: 0.35185 | train_logloss: 3.53121 | train_accuracy: 0.2996  | valid_logloss: 3.53458 | valid_accuracy: 0.3011  |  0:00:07s
epoch 4  | loss: 0.3381  | train_logloss: 2.33125 | train_accuracy: 0.3094  | valid_logloss: 2.33064 | valid_accuracy: 0.3104  |  0:00:09s
epoch 5  | loss: 0.32697 | train_logloss: 2.58933 | train_accuracy: 0.3338  | valid_logloss: 2.58154 | valid_accuracy: 0.33413 |  0:00:11s
epoch 6  | loss: 0.32382 | train_logloss: 2.30059 | train_accuracy: 0.35147 | valid_logloss: 2.29008 | valid_accuracy: 0.35547 |  0:00:13s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022


In [ ]:
time_model(number_exp=10, 
     Rows=30000, 
     Nd=128,	Na=128,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.19925 | train_logloss: 23.02097| train_accuracy: 0.3334  | valid_logloss: 23.01966| valid_accuracy: 0.33347 |  0:00:02s
epoch 1  | loss: 0.6253  | train_logloss: 23.50548| train_accuracy: 0.27513 | valid_logloss: 23.47103| valid_accuracy: 0.27693 |  0:00:04s
epoch 2  | loss: 0.4407  | train_logloss: 20.73321| train_accuracy: 0.33333 | valid_logloss: 20.80043| valid_accuracy: 0.33333 |  0:00:06s
epoch 3  | loss: 0.38829 | train_logloss: 14.62514| train_accuracy: 0.19973 | valid_logloss: 14.65985| valid_accuracy: 0.20253 |  0:00:08s
epoch 4  | loss: 0.37651 | train_logloss: 7.01168 | train_accuracy: 0.3337  | valid_logloss: 7.03836 | valid_accuracy: 0.3336  |  0:00:10s
epoch 5  | loss: 0.36318 | train_logloss: 3.60119 | train_accuracy: 0.33437 | valid_logloss: 3.61014 | valid_accuracy: 0.33423 |  0:00:12s
epoch 6  | loss: 0.35146 | train_logloss: 2.68802 | train_accuracy: 0.33497 | valid_logloss: 2.68986 | valid_accuracy: 0.33483 |  0:00:14s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


##300000 rows

In [ ]:
time_model(number_exp=11, 
     Rows=300000, 
     Nd=8,	Na=8,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=1, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.39079 | train_logloss: 2.59139 | train_accuracy: 0.24259 | valid_logloss: 2.58263 | valid_accuracy: 0.24365 |  0:00:12s
epoch 1  | loss: 0.31362 | train_logloss: 0.42557 | train_accuracy: 0.84944 | valid_logloss: 0.42434 | valid_accuracy: 0.84993 |  0:00:25s
epoch 2  | loss: 0.30036 | train_logloss: 0.29473 | train_accuracy: 0.88865 | valid_logloss: 0.29652 | valid_accuracy: 0.88872 |  0:00:38s
epoch 3  | loss: 0.29274 | train_logloss: 0.28379 | train_accuracy: 0.89345 | valid_logloss: 0.28558 | valid_accuracy: 0.89349 |  0:00:51s
epoch 4  | loss: 0.28983 | train_logloss: 0.28599 | train_accuracy: 0.89053 | valid_logloss: 0.28848 | valid_accuracy: 0.89021 |  0:01:04s
epoch 5  | loss: 0.28811 | train_logloss: 0.28661 | train_accuracy: 0.89028 | valid_logloss: 0.28839 | valid_accuracy: 0.89032 |  0:01:17s
epoch 6  | loss: 0.28422 | train_logloss: 0.27525 | train_accuracy: 0.89517 | valid_logloss: 0.27846 | valid_accuracy: 0.89443 |  0:01:30s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


In [ ]:
time_model(number_exp=12, 
     Rows=300000, 
     Nd=16,	Na=16,
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.40504 | train_logloss: 1.40503 | train_accuracy: 0.41059 | valid_logloss: 1.40505 | valid_accuracy: 0.41046 |  0:00:14s
epoch 1  | loss: 0.32269 | train_logloss: 0.42897 | train_accuracy: 0.8458  | valid_logloss: 0.42735 | valid_accuracy: 0.84628 |  0:00:28s
epoch 2  | loss: 0.30726 | train_logloss: 0.30098 | train_accuracy: 0.88776 | valid_logloss: 0.3033  | valid_accuracy: 0.88717 |  0:00:43s
epoch 3  | loss: 0.29507 | train_logloss: 0.28718 | train_accuracy: 0.89144 | valid_logloss: 0.28985 | valid_accuracy: 0.89076 |  0:00:57s
epoch 4  | loss: 0.29087 | train_logloss: 0.28015 | train_accuracy: 0.8941  | valid_logloss: 0.28348 | valid_accuracy: 0.89389 |  0:01:12s
epoch 5  | loss: 0.28837 | train_logloss: 0.28311 | train_accuracy: 0.89382 | valid_logloss: 0.28657 | valid_accuracy: 0.89334 |  0:01:26s
epoch 6  | loss: 0.2863  | train_logloss: 0.27793 | train_accuracy: 0.89406 | valid_logloss: 0.28113 | valid_accuracy: 0.89375 |  0:01:41s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


In [ ]:
time_model(number_exp=13, 
          Rows=300000, 
          Nd=64,	Na=64,	
          B=2048,	BV=512,	mB=0.7,	
          λsparse=0.001,	Nsteps=5,	γ=1.7, 
          learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
          shared=2, decision=2, 
          mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.45852 | train_logloss: 1.23316 | train_accuracy: 0.49431 | valid_logloss: 1.23162 | valid_accuracy: 0.49515 |  0:00:19s
epoch 1  | loss: 0.33774 | train_logloss: 0.43007 | train_accuracy: 0.84012 | valid_logloss: 0.42901 | valid_accuracy: 0.84103 |  0:00:38s
epoch 2  | loss: 0.32425 | train_logloss: 0.34812 | train_accuracy: 0.874   | valid_logloss: 0.34836 | valid_accuracy: 0.87473 |  0:00:58s
epoch 3  | loss: 0.31809 | train_logloss: 0.30659 | train_accuracy: 0.88468 | valid_logloss: 0.30751 | valid_accuracy: 0.88546 |  0:01:17s
epoch 4  | loss: 0.3051  | train_logloss: 0.30431 | train_accuracy: 0.88572 | valid_logloss: 0.30674 | valid_accuracy: 0.8856  |  0:01:37s
epoch 5  | loss: 0.30299 | train_logloss: 0.29393 | train_accuracy: 0.88884 | valid_logloss: 0.29667 | valid_accuracy: 0.8891  |  0:01:56s
epoch 6  | loss: 0.31306 | train_logloss: 0.31689 | train_accuracy: 0.88122 | valid_logloss: 0.31775 | valid_accuracy: 0.88101 |  0:02:16s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


##3000000 rows 

In [ ]:
time_model(number_exp=14, 
     Rows=3000000, 
     Nd=8,	Na=8,	
     B=16384,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=1, decision=2, 
     mask_type='entmax')

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


In [ ]:
time_model(number_exp=15, 
     Rows=3000000, 
     Nd=16,	Na=16,
     B=16384,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.34006 | train_logloss: 2.54978 | train_accuracy: 0.37824 | valid_logloss: 2.54953 | valid_accuracy: 0.37806 |  0:01:01s
epoch 1  | loss: 0.26049 | train_logloss: 0.64599 | train_accuracy: 0.7374  | valid_logloss: 0.64489 | valid_accuracy: 0.73758 |  0:02:03s
epoch 2  | loss: 0.25213 | train_logloss: 0.31135 | train_accuracy: 0.88725 | valid_logloss: 0.31095 | valid_accuracy: 0.88724 |  0:03:05s
epoch 3  | loss: 0.24525 | train_logloss: 0.24858 | train_accuracy: 0.91084 | valid_logloss: 0.24871 | valid_accuracy: 0.91103 |  0:04:06s
epoch 4  | loss: 0.24122 | train_logloss: 0.23694 | train_accuracy: 0.91453 | valid_logloss: 0.2372  | valid_accuracy: 0.91462 |  0:05:08s
epoch 5  | loss: 0.23854 | train_logloss: 0.23283 | train_accuracy: 0.91661 | valid_logloss: 0.23323 | valid_accuracy: 0.91673 |  0:06:10s
epoch 6  | loss: 0.23613 | train_logloss: 0.233   | train_accuracy: 0.91612 | valid_logloss: 0.23331 | valid_accuracy: 0.91625 |  0:07:12s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


In [ ]:
time_model(number_exp=53, 
          Rows=3000000, 
          Nd=64,	Na=64,	
          B=16384,	BV=512,	mB=0.7,	
          λsparse=0.001,	Nsteps=5,	γ=1.7, 
          learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
          shared=2, decision=2, 
          mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.37379 | train_logloss: 2.11995 | train_accuracy: 0.44448 | valid_logloss: 2.12328 | valid_accuracy: 0.44427 |  0:01:37s
epoch 1  | loss: 0.27536 | train_logloss: 0.79771 | train_accuracy: 0.71381 | valid_logloss: 0.79647 | valid_accuracy: 0.71398 |  0:03:14s
epoch 2  | loss: 0.2544  | train_logloss: 0.29855 | train_accuracy: 0.89117 | valid_logloss: 0.29791 | valid_accuracy: 0.89166 |  0:04:52s
epoch 3  | loss: 0.25107 | train_logloss: 0.26659 | train_accuracy: 0.90728 | valid_logloss: 0.26622 | valid_accuracy: 0.9076  |  0:06:31s
epoch 4  | loss: 0.24985 | train_logloss: 0.25267 | train_accuracy: 0.91046 | valid_logloss: 0.25263 | valid_accuracy: 0.91069 |  0:08:09s
epoch 5  | loss: 0.24604 | train_logloss: 0.23961 | train_accuracy: 0.91447 | valid_logloss: 0.23959 | valid_accuracy: 0.91468 |  0:09:47s
epoch 6  | loss: 0.2407  | train_logloss: 0.23748 | train_accuracy: 0.9154  | valid_logloss: 0.2377  | valid_accuracy: 0.9155  |  0:11:25s
epoch 7 

In [ ]:
dftn_time

,Rows,train,test,Nd,Na,B,BV,mB,λsparse,Nsteps,γ,learning rate,decay rate,decay iterations,shared,decision,mask_type,accuracy_tn,time_learn_tn,time_tn,accuracy_gb,time_learn_gb,time_gb
0,9000,9000,9000,8,8,512,128,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.888889,193.754094,55.350941,0.883778,3.483706,16.722444
1,9000,9000,9000,16,16,512,128,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.888889,222.945876,61.207308,0.883778,3.464101,17.349063
2,9000,9000,9000,64,64,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.885667,364.025402,90.066453,0.883778,3.411332,16.978554
3,9000,9000,9000,32,32,512,128,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.887333,249.533577,77.371469,0.883778,3.509229,17.331297
4,9000,9000,9000,128,128,512,128,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.888111,307.560998,90.364384,0.883778,3.475400,17.451218
5,30000,30000,30000,8,8,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,1,2,entmax,0.894067,249.859268,20.273622,0.891767,8.276289,27.093480
6,30000,30000,30000,16,16,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,2,2,entmax,0.892500,321.834888,22.630230,0.891767,8.210687,27.343578
7,30000,30000,30000,64,64,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.891300,518.142053,30.078117,0.891767,8.244918,28.102216
8,30000,30000,30000,32,32,2048,512,0.7,0.001,3,1.5,0.02,0.95,200,3,3,entmax,0.894133,253.579474,26.655121,0.891767,8.093423,27.197022
9,30000,30000,30000,128,128,2048,512,0.7,0.001,5,1.7,0.02,0.95,200,2,2,entmax,0.894433,539.947831,28.425750,0.891767,7.857594,27.867677


##Другие

In [ ]:
ones(number_exp=0, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=1, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=2, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=3, 
     Rows=30000, 
     Nd=32,	Na=32,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=4, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=5dff132182fdc6bba109ce582d8d376653122098650b1ef6a0ce91aeb3c522fc
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [ ]:
memory(number_exp=5, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.97905 | train_logloss: 12.4491 | train_accuracy: 0.37373 | valid_logloss: 12.61953| valid_accuracy: 0.37013 |  0:00:02s
epoch 1  | loss: 0.56978 | train_logloss: 14.36881| train_accuracy: 0.31703 | valid_logloss: 14.37557| valid_accuracy: 0.3213  |  0:00:04s
epoch 2  | loss: 0.46949 | train_logloss: 4.74105 | train_accuracy: 0.52543 | valid_logloss: 4.76561 | valid_accuracy: 0.52323 |  0:00:06s
epoch 3  | loss: 0.40363 | train_logloss: 5.61051 | train_accuracy: 0.30353 | valid_logloss: 5.61289 | valid_accuracy: 0.30657 |  0:00:08s
epoch 4  | loss: 0.3827  | train_logloss: 2.38747 | train_accuracy: 0.30453 | valid_logloss: 2.3809  | valid_accuracy: 0.30113 |  0:00:11s
epoch 5  | loss: 0.36719 | train_logloss: 1.9456  | train_accuracy: 0.32873 | valid_logloss: 1.95286 | valid_accuracy: 0.32877 |  0:00:13s
epoch 6  | loss: 0.36546 | train_logloss: 1.8662  | train_accuracy: 0.36957 | valid_logloss: 1.87932 | valid_accuracy: 0.3725  |  0:00:15s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb5.pkl' 'tn5.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    253.5 MiB    253.5 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    259.1 MiB      5.6 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2319.0 MiB   2059.9 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
memory(number_exp=6, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.69956 | train_logloss: 19.1368 | train_accuracy: 0.30157 | valid_logloss: 18.98184| valid_accuracy: 0.3066  |  0:00:01s
epoch 1  | loss: 0.39224 | train_logloss: 13.7197 | train_accuracy: 0.3147  | valid_logloss: 13.76676| valid_accuracy: 0.31487 |  0:00:03s
epoch 2  | loss: 0.35065 | train_logloss: 17.28233| train_accuracy: 0.31073 | valid_logloss: 17.28015| valid_accuracy: 0.31097 |  0:00:04s
epoch 3  | loss: 0.33239 | train_logloss: 9.25345 | train_accuracy: 0.26283 | valid_logloss: 9.27368 | valid_accuracy: 0.26153 |  0:00:06s
epoch 4  | loss: 0.32662 | train_logloss: 5.23646 | train_accuracy: 0.19393 | valid_logloss: 5.25429 | valid_accuracy: 0.19507 |  0:00:07s
epoch 5  | loss: 0.31889 | train_logloss: 5.17342 | train_accuracy: 0.1914  | valid_logloss: 5.18116 | valid_accuracy: 0.19033 |  0:00:09s
epoch 6  | loss: 0.31295 | train_logloss: 4.63485 | train_accuracy: 0.28783 | valid_logloss: 4.65883 | valid_accuracy: 0.2864  |  0:00:10s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb6.pkl' 'tn6.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    250.9 MiB    250.9 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    256.5 MiB      5.7 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2320.1 MiB   2063.6 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
memory(number_exp=7, 
     Rows=30000, 
     Nd=128,	Na=128,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.79359 | train_logloss: 15.44778| train_accuracy: 0.33477 | valid_logloss: 15.47836| valid_accuracy: 0.33477 |  0:00:01s
epoch 1  | loss: 0.42554 | train_logloss: 22.44025| train_accuracy: 0.33337 | valid_logloss: 22.46656| valid_accuracy: 0.33317 |  0:00:03s
epoch 2  | loss: 0.35965 | train_logloss: 12.26977| train_accuracy: 0.3281  | valid_logloss: 12.2782 | valid_accuracy: 0.3278  |  0:00:05s
epoch 3  | loss: 0.34007 | train_logloss: 5.79972 | train_accuracy: 0.3622  | valid_logloss: 5.79605 | valid_accuracy: 0.36187 |  0:00:06s
epoch 4  | loss: 0.32537 | train_logloss: 3.26044 | train_accuracy: 0.46783 | valid_logloss: 3.29715 | valid_accuracy: 0.4634  |  0:00:08s
epoch 5  | loss: 0.31751 | train_logloss: 5.39595 | train_accuracy: 0.31773 | valid_logloss: 5.41002 | valid_accuracy: 0.31917 |  0:00:09s
epoch 6  | loss: 0.31533 | train_logloss: 4.50698 | train_accuracy: 0.31837 | valid_logloss: 4.50554 | valid_accuracy: 0.3204  |  0:00:11s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb7.pkl' 'tn7.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    254.0 MiB    254.0 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    259.6 MiB      5.7 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2332.1 MiB   2072.5 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
ones(number_exp=8, 
     Rows=30000, 
     Nd=128,	Na=128,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=9, 
     Rows=30000, 
     Nd=64,	Na=64,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=7,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=20,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
memory(number_exp=10, 
     Rows=9000, 
     Nd=64,	Na=64,	
     B=512,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.04583 | train_logloss: 11.84129| train_accuracy: 0.354   | valid_logloss: 11.79447| valid_accuracy: 0.35944 |  0:00:01s
epoch 1  | loss: 0.61206 | train_logloss: 8.09429 | train_accuracy: 0.61311 | valid_logloss: 7.91257 | valid_accuracy: 0.62033 |  0:00:03s
epoch 2  | loss: 0.52071 | train_logloss: 20.06188| train_accuracy: 0.35133 | valid_logloss: 20.21294| valid_accuracy: 0.35156 |  0:00:05s
epoch 3  | loss: 0.46929 | train_logloss: 8.4903  | train_accuracy: 0.22744 | valid_logloss: 8.40832 | valid_accuracy: 0.22922 |  0:00:07s
epoch 4  | loss: 0.41851 | train_logloss: 1.92663 | train_accuracy: 0.55256 | valid_logloss: 1.9921  | valid_accuracy: 0.54578 |  0:00:09s
epoch 5  | loss: 0.41071 | train_logloss: 1.43395 | train_accuracy: 0.52244 | valid_logloss: 1.43481 | valid_accuracy: 0.53011 |  0:00:11s
epoch 6  | loss: 0.40321 | train_logloss: 1.31418 | train_accuracy: 0.51211 | valid_logloss: 1.28267 | valid_accuracy: 0.52656 |  0:00:13s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb10.pkl' 'tn10.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    255.4 MiB    255.4 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    259.8 MiB      4.4 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2319.0 MiB   2059.3 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
  number_exp=11
  Rows=30000
  Nd=64
  Na=64	
  B=512
  BV=128
  mB=0.7	
  λsparse=0.001
  Nsteps=5
  γ=1.5 
  learning_rate=0.02
  decay_rate=0.95
  decay_iterations=200	
  shared=2
  decision=2
  mask_type='entmax'

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('number: '+str(11)+', ')
  
  #data
  data_split = data_preparation(X, y, c=Rows//3, test_size=0.3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = Rows//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  #X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  #X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  #X_valid      = np.concatenate((X1_test[2*count : 3*count], X2_test[2*count : 3*count], X3_test[2*count : 3*count]))
  #y_valid      = np.concatenate((y1_test[2*count : 3*count], y2_test[2*count : 3*count], y3_test[2*count : 3*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  #X_valid_norm = robust.transform(X_valid)


  #classifier
  tn = TabNetClassifier( n_d=Nd, n_a=Na,
                          n_shared=shared, n_independent=decision,
                          momentum=mB,
                          optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=learning_rate),
                          scheduler_params={"step_size":decay_iterations, # how to use learning rate scheduler
                                            "gamma":decay_rate},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,

                          mask_type=mask_type,
                          **{ 'gamma': γ,
                              'lambda_sparse':λsparse,
                              'n_steps': Nsteps}

  )

  max_epochs = 2000
  t = time()
  tn.fit(
      X_train=X_train, y_train=y_train,
      #X_valid=X_valid, y_valid=y_valid,
      eval_set=[(X_train, y_train), (X_test, y_test)],
      eval_name=['train', 'valid'],
      eval_metric=['logloss','accuracy'],
      max_epochs=max_epochs , patience=20,
      batch_size=B, virtual_batch_size=BV,
  ) 
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time TN: '+str(t)+', ')

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'colsample_bytree': 0.6437405148446416,
    'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 100,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_test_norm, y_test)],  **lgb_fit_params)
  t = time()-t
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('time GB: '+str(t)+', ')

  #Accuracy
  acc, err = bootstrap_accuracy(tn, X_test, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc TN: '+str(acc)+'+-'+str(err)+', ')
  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(tn, 'TN', Rows)
  feature_acc(gb, 'GB', Rows)

  #save model

  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb'+str(number_exp)+'.txt')
  tn.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/tn'+str(number_exp))

In [ ]:
ones(number_exp=12, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.5, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='sparsemax')

In [ ]:
ones(number_exp=13, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=128,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
memory(number_exp=14, 
     Rows=300000, 
     Nd=16,	Na=16,	
     B=16384,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.2, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.66566 | train_logloss: 14.13323| train_accuracy: 0.25928 | valid_logloss: 14.09257| valid_accuracy: 0.26009 |  0:00:11s
epoch 1  | loss: 0.36818 | train_logloss: 8.37912 | train_accuracy: 0.3296  | valid_logloss: 8.38603 | valid_accuracy: 0.3297  |  0:00:22s
epoch 2  | loss: 0.33743 | train_logloss: 2.09891 | train_accuracy: 0.33258 | valid_logloss: 2.09696 | valid_accuracy: 0.33267 |  0:00:34s
epoch 3  | loss: 0.32329 | train_logloss: 1.56248 | train_accuracy: 0.33715 | valid_logloss: 1.5629  | valid_accuracy: 0.33734 |  0:00:45s
epoch 4  | loss: 0.31404 | train_logloss: 3.23258 | train_accuracy: 0.31048 | valid_logloss: 3.2338  | valid_accuracy: 0.31048 |  0:00:57s
epoch 5  | loss: 0.31225 | train_logloss: 2.10267 | train_accuracy: 0.38407 | valid_logloss: 2.10322 | valid_accuracy: 0.38382 |  0:01:08s
epoch 6  | loss: 0.30942 | train_logloss: 1.47126 | train_accuracy: 0.34909 | valid_logloss: 1.4706  | valid_accuracy: 0.34812 |  0:01:20s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb14.pkl' 'tn14.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    252.0 MiB    252.0 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    270.2 MiB     18.2 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2315.6 MiB   2045.4 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
ones(number_exp=15, 
     Rows=300000, 
     Nd=16,	Na=16,	
     B=16384,	BV=1024,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
memory(number_exp=20, 
     Rows=3000000, 
     Nd=16,	Na=16,	
     B=30000,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=3,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 0.38582 | train_logloss: 5.25475 | train_accuracy: 0.13576 | valid_logloss: 4.21574 | valid_accuracy: 0.17169 |  0:01:05s
epoch 1  | loss: 0.28353 | train_logloss: 1.27363 | train_accuracy: 0.30494 | valid_logloss: 1.20286 | valid_accuracy: 0.34235 |  0:02:10s
epoch 2  | loss: 0.27088 | train_logloss: 0.72128 | train_accuracy: 0.72223 | valid_logloss: 0.58551 | valid_accuracy: 0.80174 |  0:03:14s
epoch 3  | loss: 0.26378 | train_logloss: 0.38992 | train_accuracy: 0.85821 | valid_logloss: 0.32406 | valid_accuracy: 0.89515 |  0:04:19s
epoch 4  | loss: 0.26047 | train_logloss: 0.28149 | train_accuracy: 0.89758 | valid_logloss: 0.24074 | valid_accuracy: 0.9191  |  0:05:24s
epoch 5  | loss: 0.25833 | train_logloss: 0.26003 | train_accuracy: 0.905   | valid_logloss: 0.22064 | valid_accuracy: 0.92386 |  0:06:29s
epoch 6  | loss: 0.25563 | train_logloss: 0.25203 | train_accuracy: 0.90783 | valid_logloss: 0.2188  | valid_accuracy: 0.92543 |  0:07:34s
epoch 7 

In [ ]:
!python -m memory_profiler '/content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py' 'gb20.pkl' 'tn20.pkl'

Filename: /content/drive/MyDrive/Научная работа/Data/hyper/X-r/gb.py

Line #    Mem usage    Increment  Occurences   Line Contents
     7    257.2 MiB    257.2 MiB           1   @profile
     8                                         def main_func(a, b):
     9                                             #tn = TabNetClassifier()
    10                                             #tn.load_model('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)
    11                                             #gb = lgb.Booster(model_file='/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    12    302.7 MiB     45.5 MiB           1       gb = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+a)
    13   2342.0 MiB   2039.3 MiB           1       tn = joblib.load('/content/drive/MyDrive/Научная работа/Data/hyper/'+b)




In [ ]:
ones(number_exp=16, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=17, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.9,	
     λsparse=0.001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

In [ ]:
ones(number_exp=18, 
     Rows=30000, 
     Nd=16,	Na=16,	
     B=2048,	BV=512,	mB=0.7,	
     λsparse=0.0001,	Nsteps=5,	γ=1.7, 
     learning_rate=0.02,	decay_rate=0.95,	decay_iterations=200,	
     shared=2, decision=2, 
     mask_type='entmax')

Device used : cuda
epoch 0  | loss: 1.0721  | train_logloss: 20.48473| train_accuracy: 0.3116  | valid_logloss: 20.52434| valid_accuracy: 0.31487 |  0:00:02s
epoch 1  | loss: 0.62702 | train_logloss: 7.26988 | train_accuracy: 0.28423 | valid_logloss: 7.35665 | valid_accuracy: 0.28297 |  0:00:04s
epoch 2  | loss: 0.4926  | train_logloss: 4.78337 | train_accuracy: 0.3647  | valid_logloss: 4.88082 | valid_accuracy: 0.36347 |  0:00:06s
epoch 3  | loss: 0.44313 | train_logloss: 3.25561 | train_accuracy: 0.29043 | valid_logloss: 3.28029 | valid_accuracy: 0.29257 |  0:00:08s
epoch 4  | loss: 0.4382  | train_logloss: 5.81552 | train_accuracy: 0.35523 | valid_logloss: 5.85558 | valid_accuracy: 0.35263 |  0:00:10s
epoch 5  | loss: 0.42124 | train_logloss: 3.80157 | train_accuracy: 0.3198  | valid_logloss: 3.82596 | valid_accuracy: 0.32247 |  0:00:12s
epoch 6  | loss: 0.40068 | train_logloss: 2.142   | train_accuracy: 0.3114  | valid_logloss: 2.15032 | valid_accuracy: 0.31263 |  0:00:14s
epoch 7 

In [ ]:
  c1 = 9000
  data_split = data_preparation(X, y, c=c1//3)

  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = c1//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)

  gb = lgb.LGBMClassifier(  #ЛУЧШАЯ МОДЕЛЬ
    **{'colsample_bytree': 0.6437405148446416,
    'learning_rate': 0.0741521019613115,
    'min_child_samples': 9+1,
    'min_child_weight': 0.43858057836890685,
    'n_estimators': 10000,
    'num_leaves': 59+10}
  )

  t = time()
  gb.fit(X_train_norm, y_train, eval_set=[(X_train_norm, y_train), (X_valid_norm, y_valid)],  **lgb_fit_params)
  t = time()-t
  print(t)

  data_split = data_preparation(X, y, c=30000//3)


  X_train, X_test = data_split[:2]
  y_train, y_test = data_split[2:4]

  train = np.concatenate((X_train, y_train.reshape((len(y_train), 1))), axis=1)
  np.random.shuffle(train)
  X_train, y_train = train[:,:-1], train[:,-1].astype('int')

  count = 30000//3

  X1_train, X1_test = data_split[4:6]
  y1_train, y1_test = data_split[6:8]
  X2_train, X2_test = data_split[8:10] 
  y2_train, y2_test = data_split[10:12]
  X3_train, X3_test = data_split[12:14]
  y3_train, y3_test = data_split[14:16]

  X_train_pred = np.concatenate((X1_test[2*count : 4*count], X2_test[2*count : 4*count], X3_test[2*count : 4*count])) ###############
  X_val_pred   = np.concatenate((X1_test[count : 2*count], X2_test[count : 2*count], X3_test[count : 2*count]))
  np.random.shuffle(X_train_pred)
  np.random.shuffle(X_val_pred)

  X_valid      = np.concatenate((X1_test[4*count : 5*count], X2_test[4*count : 5*count], X3_test[4*count : 5*count]))
  y_valid      = np.concatenate((y1_test[4*count : 5*count], y2_test[4*count : 5*count], y3_test[4*count : 5*count]))

  robust = RobustScaler()

  X_train_norm = robust.fit_transform(X_train)
  X_test_norm = robust.transform(X_test)
  X_valid_norm = robust.transform(X_valid)


  #Accuracy
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('GB '+str(c1)+'   ')

  acc, err = bootstrap_accuracy(gb, X_test_norm, y_test)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('acc GB: '+str(acc)+'+-'+str(err)+', ')

  #Feature importance
  feature_acc(gb, 'GB', Rows)
  with open('/content/drive/MyDrive/Научная работа/Data/hyperparametrs.txt', 'a') as f:
    f.write('\n')
  gb.booster_.save_model('/content/drive/MyDrive/Научная работа/Data/hyper/gb_size'+str(c1)+'.txt')